In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy.linalg import inv
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import t

In [2]:
X = pd.read_csv("X.csv", index_col=0)
Y = pd.read_csv("Y.csv", header=None).values
X = X.drop(columns=['acq_industry_0', 'tar_industry_0'])
n = X.shape[0]
p = X.shape[1]

In [18]:
remove = np.where(Y > 2)
X = np.delete(X, remove, 0)
Y = np.delete(Y, remove, 0)

In [3]:
X = X.values
X = np.hstack((np.ones((X.shape[0],1)), X))

In [4]:
regr = linear_model.LinearRegression()
regr.fit(X, Y)

# Make predictions
Y_pred = regr.predict(X)

# Compute some statistics
res = Y - Y_pred
res = res.flatten()
#standardized residuals
sig_hat = np.sqrt(sum(res**2)/(n-p-1))
h = X.dot(inv(X.transpose().dot(X) + 0.001*np.eye(p+1))).dot(X.transpose())
h_diag = np.diag(h)
std_res = res/(sig_hat*np.sqrt(1-h_diag))

In [5]:
#predicted residuals
pred_res = res/(1-h_diag)
#RSS[i]
RSS = sum(res**2)
RSS_i = RSS - pred_res*res
#standardized predicted residuals
std_pred_res = (pred_res*np.sqrt(1 - h_diag))/np.sqrt(RSS_i/(n-p-2))
#Cook's distance
cook = std_res**2/(p+1)*h_diag/(1-h_diag)

In [41]:
# plot #
plt.figure()
plt.scatter(Y_pred, res)
plt.xlabel("fitted values")
plt.ylabel("residual")
plt.title("Residuals VS Fitted")
plt.show()

plt.scatter(Y_pred, std_res)
plt.xlabel("fitted values")
plt.ylabel("standardized residuals")
plt.title("Standardized Residuals VS Fitted")
plt.show()

plt.scatter(np.arange(n), cook)
plt.xlabel("fitted values")
plt.ylabel("Cook's distance")
plt.title("Cook's Distance VS Fitted")
plt.show()

In [6]:
degf = n-p-2
t_stat = std_pred_res
p_val = 2*t.cdf(-abs(t_stat), degf)
plt.scatter(np.arange(n), p_val)
sum(p_val<0.05)

15

In [7]:
sum(p_val<(0.05/n))
ind = np.where(p_val<(0.05/n))[0]
ind

array([ 388,  389,  499, 3473, 3831, 4009], dtype=int64)

In [8]:
X_outlier = np.delete(X, ind, 0)
Y_outlier = np.delete(Y, ind, 0)
Y_outlier = np.sqrt(Y_outlier - min(Y_outlier)+0.001)


In [9]:
regr_2 = linear_model.LinearRegression()
regr_2.fit(X_outlier, Y_outlier)
Y_pred_outlier = regr_2.predict(X_outlier)
# The mean squared error: Training error
print("Mean squared error: %.2f"
      % mean_squared_error(Y_outlier, Y_pred_outlier))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_outlier, Y_pred_outlier))
# The coefficients
print('Coefficients: \n', regr.coef_)

Mean squared error: 0.01
Variance score: 0.05
Coefficients: 
 [[  1.46869049e-11   2.32096648e-06   6.41371045e-06  -1.56654934e-05
    4.71161923e-06   1.65211797e-06   4.07795011e-06  -5.61004940e-06
   -7.21599118e-06   7.03168549e-07   9.76114968e-06  -2.67572590e-07
   -1.05221236e-06   1.59605793e-07  -1.39710652e-06   2.92041965e-06
    1.33563041e-06  -4.82771722e-06   4.18567439e-07   2.82894685e-06
   -2.19161802e-06   6.13101787e-06  -6.31208442e-06   3.41340077e-06
   -2.38515873e-06   1.76217874e-03  -2.69874152e-03   7.32414337e-04
   -2.41348504e-03   1.35635811e-03   1.31871276e-03  -8.19887985e-04
   -2.30373635e-04  -3.78882128e-04   3.23821231e-03  -1.60507217e-03
   -1.17043799e-03  -3.68542865e-06   3.46291571e-05  -3.38740827e-05
    9.88892428e-07   1.47007199e-05  -2.56515539e-05   1.02025904e-05
    9.76339022e-06  -1.44124338e-05  -2.36071380e-05   1.41177921e-05
    1.12728603e-05  -6.82129303e-02   9.24703946e-02  -5.14289221e-03
   -5.17151052e-02  -1.59245

In [81]:
res_out = Y_outlier - Y_pred_outlier
plt.scatter(Y_pred_outlier, res_out)
plt.xlabel("fitted values")
plt.ylabel("residuals")
plt.title("Residuals VS Fitted (outliers removed)")
plt.show()

In [20]:
plt.scatter(X[:,1], Y)
plt.show()

In [21]:
X.columns[14]

AttributeError: 'numpy.ndarray' object has no attribute 'columns'